In [1]:
import torch
from predict import *

In [2]:
model = load_model('weights/HERN_dock.ckpt')

In [3]:
out_file = dock('1nca_imgt.pdb', 'AAAAAAA', model)

Saved to 'outputs/AAAAAAA_docked.pdb'


In [5]:
view_pdb(out_file)

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol